In [2]:
import math
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import numpy as np

In [5]:
# ---------- SETTINGS ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset_path = "data/human_chat_1gb_readable.txt"

In [6]:
# Model hyperparameters
batch_size = 32
block_size = 256
embed_dim = 128
n_heads = 4
n_layers = 4
dropout = 0.1
learning_rate = 2e-4
num_steps = 20000

In [7]:
# ---------- LOAD DATA ----------
print("📂 Loading dataset...")
with open(dataset_path, encoding="utf-8") as f:
    text = f.read()

print(f"Dataset length: {len(text)/1e6:.2f}M characters")

# Create a simple character vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Vocab size: {vocab_size}")

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

def encode(s):
    return [stoi[c] for c in s]

def decode(tokens):
    return ''.join([itos[t] for t in tokens])

data = torch.tensor(encode(text), dtype=torch.long)

📂 Loading dataset...
Dataset length: 6.22M characters
Vocab size: 89


In [8]:
# Train/val split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    d = train_data if split == "train" else val_data
    ix = torch.randint(len(d) - block_size, (batch_size,))
    x = torch.stack([d[i:i+block_size] for i in ix])
    y = torch.stack([d[i+1:i+1+block_size] for i in ix])
    return x.to(device), y.to(device)

In [9]:
# ---------- MODEL DEFINITION ----------
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embed_dim, head_size, bias=False)
        self.query = nn.Linear(embed_dim, head_size, bias=False)
        self.value = nn.Linear(embed_dim, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        w = q @ k.transpose(-2, -1) * (C ** -0.5)
        w = w.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        w = F.softmax(w, dim=-1)
        w = self.dropout(w)
        v = self.value(x)
        out = w @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_heads):
        super().__init__()
        head_size = n_embd // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class TinyGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, embed_dim)
        self.pos_emb = nn.Embedding(block_size, embed_dim)
        self.blocks = nn.Sequential(*[Block(embed_dim, n_heads) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_emb(idx)
        pos_emb = self.pos_emb(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens=100):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_id), dim=1)
        return idx

model = TinyGPT().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


In [10]:
# ---------- TRAIN LOOP ----------
for step in tqdm(range(num_steps)):
    model.train()
    xb, yb = get_batch("train")
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if step % 500 == 0:
        model.eval()
        with torch.no_grad():
            xb, yb = get_batch("val")
            _, val_loss = model(xb, yb)
        print(f"Step {step}: train loss {loss.item():.4f}, val loss {val_loss.item():.4f}")

  0%|          | 4/20000 [00:00<53:41,  6.21it/s]  

Step 0: train loss 4.6260, val loss 4.5001


  3%|▎         | 504/20000 [00:18<11:47, 27.56it/s]

Step 500: train loss 2.2994, val loss 2.2753


  5%|▌         | 1004/20000 [00:37<12:15, 25.83it/s]

...

 95%|█████████▌| 19005/20000 [11:26<00:34, 29.01it/s]

Step 19000: train loss 1.2473, val loss 1.0631


 98%|█████████▊| 19504/20000 [11:43<00:17, 28.38it/s]

Step 19500: train loss 1.2071, val loss 1.0988


100%|██████████| 20000/20000 [12:00<00:00, 27.76it/s]


In [13]:
import os
os.makedirs("checkpoints", exist_ok=True)
torch.save(model.state_dict(), "checkpoints/human_gpt_scratch.pt")
print("✅ Model saved to checkpoints/human_gpt_scratch.pt")


✅ Model saved to checkpoints/human_gpt_scratch.pt


In [1]:
# ---------- TEST GENERATION ----------
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print("\n💬 Sample generation:")
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

NameError: name 'torch' is not defined

In [ ]:
kumar